url: https://data.cdc.gov/Case-Surveillance/United-States-COVID-19-Cases-and-Deaths-by-State-o/9mfq-cb36

In [19]:
import types 

In [1]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import psycopg2
from covid_pass import password
#from types import type(None)


In [2]:
file = "data/US_COVID-19_Cases.csv"
covid_df = pd.read_csv(file)
covid_df.head()

,submission_date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,created_at,consent_cases,consent_deaths
0,01/22/2020,CO,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree
1,01/23/2020,CO,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree
2,01/24/2020,CO,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree
3,01/25/2020,CO,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree
4,01/26/2020,CO,0,NaN,NaN,0,NaN,0,NaN,NaN,0,NaN,03/26/2020 04:22:39 PM,Agree,Agree


In [3]:
state_df = covid_df.groupby('state').mean().reset_index()
state_df.head()

,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death
0,AK,2916.114187,NaN,NaN,59.072664,0.000000,21.346021,NaN,NaN,0.290657,0.000000
1,AL,59294.768166,77799.791045,7163.437811,689.128028,148.716418,1074.408304,1472.218905,67.656716,10.470588,1.034826
2,AR,31101.352941,NaN,NaN,406.089965,50.089552,462.674740,NaN,NaN,7.048443,0.865672
3,AS,0.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000
4,AZ,92068.069204,130396.049751,1673.671642,874.629758,31.343284,2252.785467,3083.766169,146.109453,21.062284,1.890547


In [4]:
covid_df = covid_df[["submission_date", "state", "tot_cases", "tot_death"]]
covid_df.head()

,submission_date,state,tot_cases,tot_death
0,01/22/2020,CO,0,0
1,01/23/2020,CO,0,0
2,01/24/2020,CO,0,0
3,01/25/2020,CO,0,0
4,01/26/2020,CO,0,0


In [5]:
covid_df["Total_recovered"] = covid_df.tot_cases - covid_df.tot_death
covid_df.head()

,submission_date,state,tot_cases,tot_death,Total_recovered
0,01/22/2020,CO,0,0,0
1,01/23/2020,CO,0,0,0
2,01/24/2020,CO,0,0,0
3,01/25/2020,CO,0,0,0
4,01/26/2020,CO,0,0,0


In [6]:
covid_df['date'] = pd.to_datetime(covid_df['submission_date']).dt.to_period('M')
covid_df.head()

,submission_date,state,tot_cases,tot_death,Total_recovered,date
0,01/22/2020,CO,0,0,0,2020-01
1,01/23/2020,CO,0,0,0,2020-01
2,01/24/2020,CO,0,0,0,2020-01
3,01/25/2020,CO,0,0,0,2020-01
4,01/26/2020,CO,0,0,0,2020-01


In [7]:
covid_monthly_df = covid_df.groupby(["date", "state"]).mean().reset_index()
covid_monthly_df

,date,state,tot_cases,tot_death,Total_recovered
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,0.6,0.0,0.6
...,...,...,...,...,...
655,2020-11,VT,2247.2,58.0,2189.2
656,2020-11,WA,110342.0,2398.2,107943.8
657,2020-11,WI,251648.2,2178.4,249469.8
658,2020-11,WV,25649.0,467.2,25181.8


In [8]:
covid_monthly_df = covid_monthly_df.rename(columns={"state": "State", "tot_cases": "Average_cases", "tot_death": "Average_death", "Total_recovered": "Average_recovery"})
covid_monthly_df

,date,State,Average_cases,Average_death,Average_recovery
0,2020-01,AK,0.0,0.0,0.0
1,2020-01,AL,0.0,0.0,0.0
2,2020-01,AR,0.0,0.0,0.0
3,2020-01,AS,0.0,0.0,0.0
4,2020-01,AZ,0.6,0.0,0.6
...,...,...,...,...,...
655,2020-11,VT,2247.2,58.0,2189.2
656,2020-11,WA,110342.0,2398.2,107943.8
657,2020-11,WI,251648.2,2178.4,249469.8
658,2020-11,WV,25649.0,467.2,25181.8


In [9]:
covid_monthly_df['Death_percent'] = round((covid_monthly_df.Average_death/covid_monthly_df.Average_cases)*100, 2)
covid_monthly_df['Recovery_percent'] = round((covid_monthly_df.Average_recovery/covid_monthly_df.Average_cases)*100, 2)
covid_monthly_df=covid_monthly_df.dropna()
covid_monthly_df

,date,State,Average_cases,Average_death,Average_recovery,Death_percent,Recovery_percent
4,2020-01,AZ,0.6,0.0,0.6,0.00,100.00
5,2020-01,CA,1.3,0.0,1.3,0.00,100.00
17,2020-01,IL,0.9,0.0,0.9,0.00,100.00
56,2020-01,WA,1.0,0.0,1.0,0.00,100.00
64,2020-02,AZ,1.0,0.0,1.0,0.00,100.00
...,...,...,...,...,...,...,...
655,2020-11,VT,2247.2,58.0,2189.2,2.58,97.42
656,2020-11,WA,110342.0,2398.2,107943.8,2.17,97.83
657,2020-11,WI,251648.2,2178.4,249469.8,0.87,99.13
658,2020-11,WV,25649.0,467.2,25181.8,1.82,98.18


In [10]:
print(covid_monthly_df.dtypes)

date                period[M]
State                  object
Average_cases         float64
Average_death         float64
Average_recovery      float64
Death_percent         float64
Recovery_percent      float64
dtype: object


In [11]:
file = "data/US_COVID-19_Deaths.csv"
covid_stats_df = pd.read_csv(file)
covid_stats_df.head()

,State/Territory,Total Cases,Confirmed Cases,Probable Cases,Cases in Last 7 Days,Case Rate per 100000,Total Deaths,Confirmed Deaths,Probable Deaths,Deaths in Last 7 Days,Death Rate per 100000,Case Rate per 100000 in Last 7 Days,Death Rate per 100K in Last 7 Days
0,Alaska,17072,NaN,NaN,2616,2315,84,NaN,NaN,7,11,50.7,0.1
1,Alabama,199158,169266.0,29892.0,10009,4074,3026,2818.0,208.0,112,61,29.3,0.3
2,Arkansas,117360,NaN,NaN,7648,3894,2037,NaN,NaN,143,67,36.3,0.7
3,American Samoa,0,NaN,NaN,0,0,0,NaN,NaN,0,0,0.0,0.0
4,Arizona,252768,246468.0,6300.0,10288,3524,6087,5707.0,380.0,169,84,20.5,0.3


In [12]:
covid_stats_df = covid_stats_df[['State/Territory', 'Total Cases', 'Case Rate per 100000', 'Total Deaths', 'Death Rate per 100000']]
covid_stats_df.head()

,State/Territory,Total Cases,Case Rate per 100000,Total Deaths,Death Rate per 100000
0,Alaska,17072,2315,84,11
1,Alabama,199158,4074,3026,61
2,Arkansas,117360,3894,2037,67
3,American Samoa,0,0,0,0
4,Arizona,252768,3524,6087,84


In [13]:
covid_stats_df['State'] = state_df['state']
covid_stats_df.head()

,State/Territory,Total Cases,Case Rate per 100000,Total Deaths,Death Rate per 100000,State
0,Alaska,17072,2315,84,11,AK
1,Alabama,199158,4074,3026,61,AL
2,Arkansas,117360,3894,2037,67,AR
3,American Samoa,0,0,0,0,AS
4,Arizona,252768,3524,6087,84,AZ


In [14]:
covid_stats_df = covid_stats_df.rename(columns={"state": "State", "Total Cases": "Total_cases", "Case Rate per 100000": "Case_rate_per_100k", "Total Deaths": "Total_deaths", "Death Rate per 100000": "Death_rate_per_100k"})
covid_stats_df = covid_stats_df[["State", "Total_cases", "Case_rate_per_100k", "Total_deaths", "Death_rate_per_100k"]].dropna()
covid_stats_df.tail()

,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k
55,VT,2303,368,58,9
56,WA,112550,1494,2431,32
57,WI,263571,4534,2269,39
58,WV,26547,1470,480,26
59,WY,15409,2667,105,18


In [39]:
engine = create_engine('sqlite:///covid.db')

covid_stats_df.to_sql(name='covid_total', con=engine, if_exists='replace', index=False)
covid_monthly_df.to_sql(name='covid_monthly_avg', con=engine, if_exists='replace', index=False)
#covid_monthly_df.to_sql('covid_monthly_avg', con=engine, if_exists='replace', index=False)

## testing
# df = pd.DataFrame({'a': [1,2],
# 'b': [3,4]})
# df.to_sql('test', con=engine)
# pd.read_sql('select * from test', engine)
# covid_monthly_df['date'] = covid_monthly_df['date'].astype(str)
# covid_monthly_df.dtypes

In [40]:
pd.read_sql('select * from covid_total', engine)


,State,Total_cases,Case_rate_per_100k,Total_deaths,Death_rate_per_100k
0,AK,17072,2315,84,11
1,AL,199158,4074,3026,61
2,AR,117360,3894,2037,67
3,AS,0,0,0,0
4,AZ,252768,3524,6087,84
5,CA,944576,2388,17815,45
6,CO,121006,2125,2353,41
7,CT,77060,2157,4656,130
8,DC,17682,2517,650,92
9,DE,25753,2663,716,74
